In [ ]:
import lammps

In [ ]:
lmp = lammps.lammps()

In [ ]:
print("LAMMPS version: "+str(lmp.version()))
print("OS info: "+lmp.get_os_info())

In [ ]:
# Some auxiliary functions
def lmp_read_data(lmp, data_file) :
    lmp.command("read_data "+data_file)

In [ ]:
!ls

In [ ]:
# Rename or make a fancy function to 'guess' the file types
in_file = "Water_Graph_40%_relax.in"
init_file = "Water_Graph_40%_relax.in.init"
data_file = "Water_Graph_40%.data"
settings_file = "Water_Graph_40%_relax.in.settings"
run_file = "Water_Graph_40%_relax.in.run"
ff_files = {"TERSOFF_forcefield.ff"}

In [ ]:
!cat README.txt

In [ ]:
# We could just read this file with "lmp.file()", but let's go step-by-step
!cat {in_file}

### Initialization

In [ ]:
!cat {init_file}

In [ ]:
# Reading initialization file:
lmp.file(init_file)

### Data (topology and configuration)

In [ ]:
!cat {data_file}

In [ ]:
# Reading data file:
lmp_read_data(lmp,data_file)

### Simulation settings and force field

In [ ]:
# Settings
!cat {settings_file}

In [ ]:
for ff in ff_files:
    print(ff+":\n")
    !cat {ff}
    print("\n")

In [ ]:
# It would be better to pass the force field as variable, but I don't know if that's possible
lmp.file(settings_file)

### Run

In [ ]:
!cat {run_file}

In [ ]:
# Runs parameters:
dump_thermodynamics = 100
dump_trajectory = 100
time_step = 1.0 # [fs]
velocity_seed = 391624
temperature_1 = 100 # [K]
langevin_seed = 537199
deformation_freq = 1
deformation_scale = 0.957
n_steps = 1000

In [ ]:
# Output files
out_file_1 = "compressed_water-graph_40%.data"

In [ ]:
# Physical observables
lmp.commands_list([
    "compute temp_water water temp",
    "compute temp_graph graph temp",
    "compute msqdis  all msd com yes"
])

In [ ]:
# Output.
lmp.commands_list([
    "thermo "+str(dump_thermodynamics),
    """thermo_style custom step etotal evdwl ecoul elong ebond eangle ke pe temp c_temp_water c_temp_graph &
    press vol density c_msqdis[1] c_msqdis[2] c_msqdis[3] c_msqdis[4] cpu""",
    "thermo_modify flush yes"
])

In [ ]:
# Save UNWRAPPED coordinates.
lmp.commands_list([
    "dump 1 all custom "+str(dump_trajectory)+" ${run}.dump id type element x y z xu yu zu vx vy vz mol",
    "dump_modify 1 element O H C C O H"
])

In [ ]:
# Fixes for: 1. Shrink the cell to the target density.
lmp.commands_list([
    "timestep "+str(time_step),
    "velocity water create ${temp} "+str(velocity_seed)+" dist gaussian",
    "fix freeze graph setforce 0.0 0.0 0.0",
    "fix 3 water nve",
    "fix 4 water langevin ${temp} ${temp} "+str(temperature_1)+" "+str(langevin_seed),
    "fix_modify 4 temp temp_water", 
    "fix 5 water deform "+str(deformation_freq)+" z scale "+str(deformation_scale)+" units box remap x"
])

In [ ]:
"""
for i in range(n_cycles) :
    print("*** Shrinking step ",i,"/",n_cycles," ***")
    lmp.command("run "+str(n_steps_per_cycle))
"""

In [ ]:
# Very short run
lmp.command("run "+str(n_steps))

In [ ]:
# Output
lmp.command("write_data "+out_file_1+" nocoeff")

In [ ]:
lmp.close()